In [1]:
import os
import matplotlib.pyplot as plt

from pathlib import Path
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

from spikesorters.sorterlist import sorter_dict
from utils import prepare_hungarian_agreement_dataset
import spikesorters as ss

os.environ["TMP"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TEMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["ML_TEMPORARY_DIRECTORY"] = "/home/mclancy/SpikeConfidence/.tmp"

base_dir = Path('/home/mclancy/SpikeConfidence/')
analyses_dir = base_dir / 'analyses'
spike_sorter_dir = base_dir / 'spikesorters'

# # kilosort2
# !git clone https://github.com/MouseLand/Kilosort2.git
# kilosort2_path = spike_sorter_dir / 'Kilosort2.5'
# ss.Kilosort2Sorter.set_kilosort2_path(kilosort2_path)

ss.IronClustSorter.set_ironclust_path(spike_sorter_dir / 'ironclust')

Setting IRONCLUST_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/ironclust


In [ ]:
dataset_path = Path("/home/mclancy/SpikeConfidence/analyses/recordings_50cells_SqMEA_2020/recordings_50cells_SqMEA-10-15_600.0_10.0uV_21-01-2020_18-12.h5")
# session = SpikeSorter(dataset_path)
# recording = session.get_recording()
# ground_truth = session.get_ground_truth()
#
#
# sorting = session.get_sorting(sorter_name='herdingspikes')

# sorter_names = ['klusta', 'tridesclous', 'mountainsort4',
#                 'ironclust', 'kilosort', 'kilosort2', 'kilosort2_5',
#                 'waveclus']

sorter_names = ['herdingspikes', 'spykingcircus', 'ironclust', 'tridesclous', 'mountainsort4']

metric_names = ["num_spikes", "firing_rate", "presence_ratio",
                "isi_violation", "amplitude_cutoff", "snr",
                "max_drift", "cumulative_drift", "silhouette_score",
                "isolation_distance", "l_ratio",
                "nn_hit_rate", "nn_miss_rate"] #"d_prime",

X, y = prepare_hungarian_agreement_dataset(dataset_path=dataset_path, sorter_names=sorter_names, metric_names=metric_names)

/home/mclancy/.local/share/pyenv/versions/SpikeConfidence/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Running mountainsort4...


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [13]:
linear_glm = sm.GLM(y_train, X_train, family=sm.families.Binomial())
linear_result = linear_glm.fit()

In [14]:
y_test_preds = linear_glm.predict(X_test)

# plt.plot(y, linear_result.predict(X), 'o', alpha=0.2);


ValueError: shapes (112,13) and (56,13) not aligned: 13 (dim 1) != 56 (dim 0)

In [ ]:
print(X_train)